In [1]:
from lmfit import Parameters
from lmfit import minimize, fit_report

In [2]:
import pyroll.core as pr
import pyroll.pillar_model
import pyroll.neutral_point_estimator
import pyroll.sparling_spreading
import pyroll.hensel_force_torque
import pyroll.freiberg_flow_stress
import pyroll.ring_model_thermal

In [3]:
from in_profile import create_in_profile_from_surface_temperature
from rolling_sequence import rolling_train_e112, rolling_train_e107
from utilities import *

In [4]:
pyroll.pillar_model.Config.PILLAR_COUNT = 30
pyroll.neutral_point_estimator.Config.ESTIMATOR = "EQUAL"
ROLL_SURFACE_TEMPERATURE = 50 + 273.15
DISK_ELEMENT_COUNT = 30

In [5]:
@pr.CoolingPipe.DiskElement.heat_transfer_coefficient
def heat_transfer_coefficient_model_moreales(self: pr.CoolingPipe.DiskElement):
    mean_surface_temperature = (self.in_profile.surface_temperature + self.out_profile.surface_temperature) / 2
    profile_surface_temperature_celsius = mean_surface_temperature - 273.15
    return 13950 + 5.12 * 1e6 / profile_surface_temperature_celsius

In [6]:
in_profile = create_in_profile_from_surface_temperature(1050 + 273.15)
in_profile.ring_temperatures = in_profile.ring_temperatures

In [7]:
rolling_train_e112 = rolling_train_e112(ROLL_SURFACE_TEMPERATURE, DISK_ELEMENT_COUNT)
rolling_train_e107 = rolling_train_e107(ROLL_SURFACE_TEMPERATURE, DISK_ELEMENT_COUNT)

In [8]:
rolling_train_e112.solve(in_profile)

Solution iteration of TwoRollPass '3' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '7' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '14' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '18-V' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '23-H' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '18-V' exceeded the maximum iteration count of 100. Continuing anyway.


Profile(
    classifiers={'false_round', 'generic_elongation', 'round', 'symmetric'},
    core_temperature=np.float64(1570.0591539343973),
    cross_section=Polygon(
        area=5.939031536528554e-05,
        height=0.008700000000000006,
        perimeter=0.027369332464370438,
        width=0.008768227336460339,
        ),
    length=5171.222724142008,
    material=['BST500', 'steel'],
    ring_temperatures=array([1570.05915393, 1567.12701904, 1558.34092613, 1543.71864333,
           1523.23735286, 1496.84675482, 1464.74408402, 1428.15944014,
           1390.54359123, 1358.13149   , 1337.87659106]),
    strain=np.float64(0.5607017311212023),
    surface_temperature=np.float64(1337.876591055721),
    t=np.float64(36.875179665279155),
    temperature=np.float64(1425.8613020468924),
    weight=2400,
    width=np.float64(0.008768227336460339),
    )

In [9]:
rolling_train_e107.solve(in_profile)

Solution iteration of TwoRollPass '3' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '7' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '14' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '22-V' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '23-H' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of TwoRollPass '27-H' exceeded the maximum iteration count of 100. Continuing anyway.


Profile(
    classifiers={'false_round', 'generic_elongation', 'round', 'symmetric'},
    core_temperature=np.float64(1564.031451573547),
    cross_section=Polygon(
        area=5.896233250690301e-05,
        height=0.008700000000000006,
        perimeter=0.027243894306999205,
        width=0.008532481593459252,
        ),
    length=5208.739922328763,
    material=['BST500', 'steel'],
    ring_temperatures=array([1564.03145157, 1561.14151648, 1552.47714682, 1538.03928781,
           1517.78882721, 1491.71342657, 1460.18475728, 1424.7379923 ,
           1389.0032269 , 1358.83558383, 1340.17996623]),
    strain=np.float64(0.5201991307214924),
    surface_temperature=np.float64(1340.1799662253916),
    t=np.float64(36.85417501903466),
    temperature=np.float64(1423.7227334780737),
    weight=2400,
    width=np.float64(0.008532481593459252),
    )

In [10]:
def solve_total_wear_depths_as_contour_line(params):
    sequences = [
        {
            "label": "e11_2",
            "sequence": rolling_train_e112
        },
        {
            "label": "e10_7",
            "sequence": rolling_train_e107
        }
    ]

    area_errors = []

    for entry in sequences:
        groove_label = entry["label"]
        sequence = entry["sequence"]
        roll_passes = sequence.roll_passes
        measurements = extract_wear_contours_from_measurement(groove_label)

        for measurement_id, labels, tonnages, measured_wear_contours in measurements:
            for roll_pass in roll_passes:
                for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                    if roll_pass.label == label:

                        wear_coefficient = get_wear_coefficient(roll_pass, params)
                        calculated_wear_contour = calculate_wear_contour(roll_pass, wear_coefficient, tonnage * 1000)
                        area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)
                        area_errors.append(area_error)
                    else:
                        continue
    rms_error = root_mean_square_value(np.array(area_errors))
    return rms_error


In [30]:
def solve_total_wear_depths_as_contour_line_4_pass_block(params):
    sequences = [
        {
            "label": "e11_2",
            "sequence": rolling_train_e112
        },
        {
            "label": "e10_7",
            "sequence": rolling_train_e107
        }
    ]

    area_errors = []

    for entry in sequences:
        groove_label = entry["label"]
        sequence = entry["sequence"]
        roll_passes = sequence.roll_passes
        measurements = extract_wear_contours_from_measurement(groove_label)

        for measurement_id, labels, tonnages, measured_wear_contours in measurements:
            for roll_pass in roll_passes:
                for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                    if roll_pass.label == label and label in ["17-H", "18-V"]:

                        wear_coefficient = get_wear_coefficient(roll_pass, params)
                        calculated_wear_contour = calculate_wear_contour(roll_pass, wear_coefficient, tonnage * 1000)
                        area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)
                        area_errors.append(area_error)
                    else:
                        continue
    rms_error = root_mean_square_value(np.array(area_errors))
    return rms_error


def solve_total_wear_depths_as_contour_line_6_pass_block(params):
    sequences = [
        {
            "label": "e11_2",
            "sequence": rolling_train_e112
        },
        {
            "label": "e10_7",
            "sequence": rolling_train_e107
        }
    ]

    area_errors = []

    for entry in sequences:
        groove_label = entry["label"]
        sequence = entry["sequence"]
        roll_passes = sequence.roll_passes
        measurements = extract_wear_contours_from_measurement(groove_label)

        for measurement_id, labels, tonnages, measured_wear_contours in measurements:
            for roll_pass in roll_passes:
                for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                    if roll_pass.label == label and label in ["21-H", "22-V", "23-H", "24-V"]:

                        wear_coefficient = get_wear_coefficient(roll_pass, params)
                        calculated_wear_contour = calculate_wear_contour(roll_pass, wear_coefficient, tonnage * 1000)
                        area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)
                        area_errors.append(area_error)
                    else:
                        continue
    rms_error = root_mean_square_value(np.array(area_errors))
    return rms_error


def solve_total_wear_depths_as_contour_line_csb_block(params):
    sequences = [
        {
            "label": "e11_2",
            "sequence": rolling_train_e112
        },
        {
            "label": "e10_7",
            "sequence": rolling_train_e107
        }
    ]

    area_errors = []

    for entry in sequences:
        groove_label = entry["label"]
        sequence = entry["sequence"]
        roll_passes = sequence.roll_passes
        measurements = extract_wear_contours_from_measurement(groove_label)

        for measurement_id, labels, tonnages, measured_wear_contours in measurements:
            for roll_pass in roll_passes:
                for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                    if roll_pass.label == label and label in ["27-H", "28-V"]:

                        wear_coefficient = get_wear_coefficient(roll_pass, params)
                        calculated_wear_contour = calculate_wear_contour(roll_pass, wear_coefficient, tonnage * 1000)
                        area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)
                        area_errors.append(area_error)
                    else:
                        continue
    rms_error = root_mean_square_value(np.array(area_errors))
    return rms_error

In [12]:
params_constant_cobyla = Parameters()
params_constant_cobyla.add('wear_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)

params_linear_function_cobyla = Parameters()
params_linear_function_cobyla.add('base_wear_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)
params_linear_function_cobyla.add('roll_gap_ratio_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)

params_cubic_function_cobyla = Parameters()
params_cubic_function_cobyla.add('base_wear_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)
params_cubic_function_cobyla.add('roll_gap_ratio_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)
params_cubic_function_cobyla.add('speed_ratio_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)

In [14]:
result_constant_cobyla = minimize(
    solve_total_wear_depths_as_contour_line,
    params_constant_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

result_linear_function_cobyla = minimize(
    solve_total_wear_depths_as_contour_line,
    params_linear_function_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

result_cubic_function_cobyla = minimize(
    solve_total_wear_depths_as_contour_line,
    params_cubic_function_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

In [15]:
print(fit_report(result_constant_cobyla))

[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 21
    # data points      = 1
    # variables        = 1
    chi-square         = 4.8089e-10
    reduced chi-square = 4.8089e-10
    Akaike info crit   = -19.4553802
    Bayesian info crit = -21.4553802
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    wear_coefficient:  8.7963e-10 (init = 1e-06)


In [16]:
print(fit_report(result_linear_function_cobyla))

[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 29
    # data points      = 1
    # variables        = 2
    chi-square         = 4.1440e-10
    reduced chi-square = 4.1440e-10
    Akaike info crit   = -17.6041785
    Bayesian info crit = -21.6041785
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    base_wear_coefficient:       2.7404e-10 (init = 1e-06)
    roll_gap_ratio_coefficient:  1.4988e-10 (init = 1e-06)


In [17]:
print(fit_report(result_cubic_function_cobyla))

[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 46
    # data points      = 1
    # variables        = 3
    chi-square         = 5.1441e-10
    reduced chi-square = 5.1441e-10
    Akaike info crit   = -15.3880091
    Bayesian info crit = -21.3880091
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    base_wear_coefficient:       2.5760e-10 (init = 1e-06)
    roll_gap_ratio_coefficient:  2.2668e-10 (init = 1e-06)
    speed_ratio_coefficient:     1.3637e-05 (init = 1e-06)


In [19]:
params_4_pass_block_cobyla = Parameters()
params_4_pass_block_cobyla.add('wear_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)

params_6_pass_block_cobyla = Parameters()
params_6_pass_block_cobyla.add('wear_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)

params_csb_block_cobyla = Parameters()
params_csb_block_cobyla.add('wear_coefficient', value=1e-6, min=1e-10, max=1e-1, vary=True)


result_4_pass_block_cobyla = minimize(
    solve_total_wear_depths_as_contour_line_4_pass_block,
    params_4_pass_block_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

result_6_pass_block_cobyla = minimize(
    solve_total_wear_depths_as_contour_line_6_pass_block,
    params_6_pass_block_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

result_csb_block_cobyla = minimize(
    solve_total_wear_depths_as_contour_line_csb_block,
    params_csb_block_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

In [20]:
print(fit_report(result_4_pass_block_cobyla))
print(fit_report(result_6_pass_block_cobyla))
print(fit_report(result_csb_block_cobyla))

[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 21
    # data points      = 1
    # variables        = 1
    chi-square         = 1.0659e-09
    reduced chi-square = 1.0659e-09
    Akaike info crit   = -18.6594551
    Bayesian info crit = -20.6594551
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculate uncertainties
    and numdifftools is not installed for lmfit to do this. Use
    `pip install numdifftools` for lmfit to estimate uncertainties
    with this fitting method.
[[Variables]]
    wear_coefficient:  2.0126e-09 (init = 1e-06)
[[Fit Statistics]]
    # fitting method   = COBYLA
    # function evals   = 20
    # data points      = 1
    # variables        = 1
    chi-square         = 4.7699e-10
    reduced chi-square = 4.7699e-10
    Akaike info crit   = -19.4635352
    Bayesian info crit = -21.4635352
##  Warning: uncertainties could not be estimated:
    this fitting method does not natively calculat